In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt
import time
start_time = time.time()

test_data = pd.read_csv(r"D:\Downloads\Germany_Data_28-02_03-03.csv")
sarimax_predictions = pd.read_csv(r"D:\Downloads\SARIMAX_Prediction_Data_28-02-2023-05-00_03-03-2023-05-00.csv")
test_data['Datetime'] = pd.to_datetime(test_data['Datetime'])
sarimax_predictions['Datetime'] = pd.to_datetime(sarimax_predictions['Datetime'])

data = test_data.merge(
    sarimax_predictions, 
    on='Datetime', 
    suffixes=('_actual', '_predicted')
)
data.rename(columns={'Price (EUR/MWHE)': 'market_price', 'Predicted Price': 'market_price_predicted'}, inplace=True)

scaler = MinMaxScaler()
data[['market_price', 'market_price_predicted']] = scaler.fit_transform(data[['market_price', 'market_price_predicted']])

battery_capacity = 55  
initial_budget = 100  
starting_soc = 50  
charge_efficiency = 0.95
discharge_efficiency = 0.95
gamma = 0.875
soc_limits = [10, 90]  

soc = starting_soc
features, targets = [], []
for t in range(len(data) - 1):
    soc_kwh = (soc / 100) * battery_capacity
    current_price = data['market_price'].iloc[t]
    predicted_price = data['market_price_predicted'].iloc[t + 1]

    future_price = data['market_price_predicted'].iloc[min(t + 5, len(data) - 1)]
    long_term_charge_profit = gamma * (future_price - current_price) * charge_efficiency
    long_term_discharge_profit = gamma * (current_price - future_price) * discharge_efficiency
    
    charge_value = (current_price - predicted_price) * charge_efficiency + long_term_charge_profit - gamma * long_term_discharge_profit 
    discharge_value = (predicted_price - current_price) * discharge_efficiency + long_term_discharge_profit - gamma * long_term_charge_profit
    
    if 0.95 < predicted_price / current_price < 1.05:
        hold_value = 0.1 * current_price
    else:
        hold_value = -0.1 * current_price
    
    features.append([soc, current_price, predicted_price])
    targets.append([charge_value, discharge_value, hold_value])

features = np.array(features)
targets = np.array(targets)

X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.2, random_state=42)

model = Sequential([
    Dense(64, activation='relu', input_dim=3),
    Dense(64, activation='relu'),
    Dense(3, activation='linear')  
])
model.compile(optimizer='adam', loss='mse')
model.fit(X_train, y_train, epochs=70, batch_size=1, verbose=1)

predictions = model.predict(X_test)

soc = starting_soc
budget = initial_budget
charge_indices, discharge_indices, hold_indices = [], [], []
soc_values, profits = [soc], [0]
dod_values = []

for t in range(len(data) - 1):
    current_price = data['market_price'].iloc[t]
    predicted_price = data['market_price_predicted'].iloc[t + 1]
    features = np.array([[soc, current_price, predicted_price]])
    predicted_values = model.predict(features)[0]

    action_idx = np.argmax(predicted_values)
    if action_idx == 0: 
        if soc < soc_limits[1]:
            charge_amount = min((soc_limits[1] - soc) / 100 * battery_capacity, 5)  
            budget -= current_price * charge_amount / charge_efficiency
            soc += (charge_amount * charge_efficiency / battery_capacity) * 100
            charge_indices.append(t)
    elif action_idx == 1:  
        if soc > soc_limits[0]:
            discharge_amount = min((soc - soc_limits[0]) / 100 * battery_capacity, 5) 
            budget += current_price * discharge_amount / discharge_efficiency
            soc -= (discharge_amount * discharge_efficiency / battery_capacity) * 100
            discharge_indices.append(t)
            dod_values.append(discharge_amount / battery_capacity * 100)
    else: 
        hold_indices.append(t)
    
    soc = max(soc_limits[0], min(soc_limits[1], soc))
    soc_values.append(soc)
    profits.append(budget - initial_budget)
    
end_time = time.time()
execution_time = end_time - start_time
print(f"Execution Time: {execution_time:.2f} seconds")

if dod_values:
    average_dod = np.mean(dod_values)
    max_dod = np.max(dod_values)
    print(f"Average Depth of Discharge (DoD): {average_dod:.2f}%")
    print(f"Highest Depth of Discharge (DoD): {max_dod:.2f}%")
else:
    print("No discharges occurred, so Depth of Discharge (DoD) metrics are unavailable.")

plt.figure(figsize=(12, 6))
plt.plot(range(len(data)), data["market_price"], label="Market Price")
plt.scatter(charge_indices, data["market_price"].iloc[charge_indices], color="green", label="Charge")
plt.scatter(discharge_indices, data["market_price"].iloc[discharge_indices], color="red", label="Discharge")
plt.plot(range(len(data)), data["market_price_predicted"], label="Predicted Price", linestyle=":")
plt.xlabel("Index")
plt.ylabel("Price (€/kWh)")
plt.legend(loc=1)
plt.grid()
plt.show()

plt.figure(figsize=(12, 6))
plt.plot(range(len(profits)), profits, label="Cumulative Profit (€)", color="orange")
plt.axhline(0, color="red", linestyle="--", label="Break-even")
plt.xlabel("Index")
plt.ylabel("Profit (€)")
plt.legend(loc=1)
plt.grid()
plt.show()


print(f"Final budget: €{budget:.2f}")
print(f"Net profit: €{budget - initial_budget:.2f}")